# Overview

This notebook aggregates country-level carbon storage data into IPCC region definitions

In [1]:
import pandas as pd
from pandas_indexing import ismatch

from pathlib import Path

c:\Users\gidden\Miniconda3\envs\pyam\Lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
data_path = Path('../data/packaged')
write_path = Path('../data/derived')

In [3]:
def cols_to_dict(df, col1, col2):
    keys = df[col1].unique()
    df = df.set_index(col1)
    return {key: df[col2][key].to_list() for key in keys}

mdf = pd.read_csv(data_path / 'iso3c_region_mapping_20240319_highlighted.csv')
r5_mapping = cols_to_dict(mdf.dropna(subset='r5_iamc'), 'r5_iamc', 'iso3c')
r10_mapping = cols_to_dict(mdf.dropna(subset='r10_iamc'), 'r10_iamc', 'iso3c')
w_mapping = {'World': mdf.iso3c.unique()}

mdf.head()

,iso3c,m49code,r5_iamc,r6_ipcc,r10_unif,r10_iamc,r10_ipcc,r10_message,r11_message,r12_message,...,m49lo_ipcc,m49dev_ipcc,m49sids_ipcc,m49lldc_ipcc,m49ldc_ipcc,annexi_ipcc,wmo_ipcc,eu_ipcc,oecd_ipcc,wb_ipcc
0,AGO,24.0,R5MAF,Africa,R10AFRMEA,R10AFRICA,Africa,Sub-saharan Africa,Sub-saharan Africa,R12AFR,...,Middle Africa,Developing,NaN,NaN,LDC,Non-Annex I,I,NaN,NaN,Lower-middle
1,ARE,784.0,R5MAF,Middle East,R10AFRMEA,R10MIDDLE_EAST,Middle East,Middle East and North Africa,Middle East and North Africa,R12MEA,...,Western Asia,Developing,NaN,NaN,NaN,Non-Annex I,II,NaN,NaN,High
2,ATF,260.0,NaN,Africa,R10AFRMEA,NaN,Africa,NaN,NaN,NaN,...,Eastern Africa,Developing,NaN,NaN,NaN,Territory,NaN,NaN,NaN,NaN
3,BDI,108.0,R5MAF,Africa,R10AFRMEA,R10AFRICA,Africa,Sub-saharan Africa,Sub-saharan Africa,R12AFR,...,Eastern Africa,Developing,NaN,LLDC,LDC,Non-Annex I,I,NaN,NaN,Low
4,BEN,204.0,R5MAF,Africa,R10AFRMEA,R10AFRICA,Africa,Sub-saharan Africa,Sub-saharan Africa,R12AFR,...,Western Africa,Developing,NaN,NaN,LDC,Non-Annex I,I,NaN,NaN,Lower-middle


In [4]:
df = (
    pd.read_csv(data_path / 'Analysis_dataset.csv', index_col='ISO')
    .assign(
        Pot_Baseline=lambda _df: _df['Pot_OFF_Baseline'] + _df['Pot_ON_Baseline'],
        Pot_Final=lambda _df: _df['Pot_OFF_Final'] + _df['Pot_ON_Final'],
    )
    .drop(['Absolute_Loss', 'Percentage_Loss'], axis=1)
)
df.head()

,Pot_OFF_Baseline,Pot_OFF_Final,Pot_ON_Baseline,Pot_ON_Final,Pot_Baseline,Pot_Final
ISO,,,,,,
AGO,31.862534,2.326811,33.128390,8.769271e+00,64.990924,11.096082
ALB,2.926913,0.330340,1.721749,8.894249e-05,4.648661,0.330429
ARE,12.986295,2.390857,15.931671,2.831068e+00,28.917966,5.221924
ARG,103.381188,14.210426,112.580486,2.299054e+01,215.961674,37.200966
ATG,0.874872,0.071165,0.022531,1.660673e-08,0.897403,0.071165


In [5]:
def aggregate(df, mapping):
    df = df.pix.aggregate(ISO=mapping, mode='return')
    df['Percentage lost (net vs gross)'] = 1 - df['Pot_Final'] / df['Pot_Baseline']
    return df.rename_axis(index={'ISO': 'Region'})

data = pd.concat([
    aggregate(df, r5_mapping),
    aggregate(df, r10_mapping),
    aggregate(df, w_mapping),
])

data

,Pot_OFF_Baseline,Pot_OFF_Final,Pot_ON_Baseline,Pot_ON_Final,Pot_Baseline,Pot_Final,Percentage lost (net vs gross)
Region,,,,,,,
R5ASIA,1027.916959,112.481314,606.177527,53.782506,1634.094487,166.263820,0.898253
R5LAM,751.380193,86.262864,706.138026,118.429142,1457.518219,204.692007,0.859561
R5MAF,787.882298,63.768373,1562.040478,371.755321,2349.922776,435.523694,0.814665
R5OECD90+EU,2359.228057,184.726898,1169.834328,206.953153,3529.062385,391.680051,0.889013
R5REF,774.148816,26.625365,1472.801306,278.874828,2246.950121,305.500192,0.864038
R10AFRICA,687.783298,48.865328,1244.365081,314.457745,1932.148379,363.323074,0.811959
R10CHINA+,199.282501,40.993199,331.681824,37.471901,530.964324,78.465100,0.852222
R10EUROPE,805.017531,44.928517,159.313876,4.596934,964.331407,49.525451,0.948643
R10INDIA+,401.618365,15.730531,160.181381,1.743526,561.799746,17.474057,0.968896


In [6]:
data.to_csv(write_path / '101_Analysis_dataset_r5_r10.csv', index=True)